In [28]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from datetime import datetime, timedelta, date
import time
import pandas as pd

In [54]:
DATE_FROM = '20221001'
DATE_TO = '20221215'

In [59]:
def prepare_datestrings():
    '''returns a list of datestrings in format YYYYMMDD-YYYYMMDD where both start and end date correspond to consecutive fridays'''
    end_date = date(2022, 12, 8)
    datestrings = []
    while(True):
        start_date = end_date - timedelta(days = 6)
        if(end_date < date(2022, 10, 1)):
            break
        else:
            datestrings.append(f"{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}")
            end_date = end_date - timedelta(days = 7)
    return datestrings

In [61]:
datestrings = prepare_datestrings()
print(datestrings)
# get_data(datestrings)
# yt_data = load_data_into_df()
# yt_data.to_csv('./YouTube_data{DATE_FROM}-{DATE_TO}.tsv', sep='\t')

['20221202-20221208', '20221125-20221201', '20221118-20221124', '20221111-20221117', '20221104-20221110', '20221028-20221103', '20221021-20221027', '20221014-20221020', '20221007-20221013', '20220930-20221006']


In [56]:
def get_data(datestrings):
#     !rm ~/Downloads/youtube-charts-top-songs-fr-weekly*
    driver = webdriver.Chrome()
    driver.maximize_window()
    for datestr in datestrings:
        driver.get(f'https://charts.youtube.com/charts/TopSongs/fr/{datestr}?hl=pl')
        time.sleep(1)
        download_elm = driver.find_element("xpath", ".//*[@id='download-button']")
        driver.execute_script("arguments[0].click();", download_elm)
        time.sleep(1)
    driver.quit()

In [57]:
def load_data_into_df():
    column_names = ['Rank', 'Previous Rank', 'Track Name', 'Artist Names', 'Weeks on Chart', 'Views',
                    'Weekly Growth', 'YouTube URL']
    df = pd.DataFrame(columns = column_names)
    file_list = !ls ~/Downloads/youtube-charts-top-songs-fr-weekly*
    for file_name in file_list:
        df = pd.concat([df, pd.read_csv(file_name)])
    df = df.drop(columns = ['Rank', 'Previous Rank', 'Weekly Growth', 'YouTube URL', 'Weeks on Chart'])
    df = df.groupby(['Track Name', 'Artist Names'], axis=0, as_index=False).sum()
    df = df.sort_values('Views', ascending = False, ignore_index = True)
    return df